In [1]:
import numpy as np
import pandas as pd
import lightgbm  as lgb

In [2]:
train = pd.read_csv('../../csv/titanic/train.csv')
test = pd.read_csv('../../csv/titanic/test.csv')
submission = pd.read_csv('../../csv/titanic/gender_submission.csv')
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
train = pd.get_dummies(train, columns=['Sex', 'Embarked'])
test = pd.get_dummies(test, columns=['Sex', 'Embarked'])

train = train.drop(['PassengerId', 'Name', 'Cabin', 'Ticket'], axis=1)
test = test.drop(['PassengerId', 'Name', 'Cabin', 'Ticket'], axis=1)

In [4]:
X_train = train.drop(['Survived'], axis=1)
y_train = train['Survived']

In [9]:
test

,Pclass,Age,SibSp,Parch,Fare,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
0,3,34.5,0,0,7.8292,0,1,0,1,0
1,3,47.0,1,0,7.0000,1,0,0,0,1
2,2,62.0,0,0,9.6875,0,1,0,1,0
3,3,27.0,0,0,8.6625,0,1,0,0,1
4,3,22.0,1,1,12.2875,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...
413,3,NaN,0,0,8.0500,0,1,0,0,1
414,1,39.0,0,0,108.9000,1,0,1,0,0
415,3,38.5,0,0,7.2500,0,1,0,0,1
416,3,NaN,0,0,8.0500,0,1,0,0,1


In [5]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [6]:
train_X, valid_X, train_y, valid_y = train_test_split(X_train, y_train, test_size=0.33, random_state=42)

gbm = lgb.LGBMClassifier(objective='binary')
gbm.fit(
    train_X, 
    train_y,
    eval_set=[(valid_X, valid_y)],
    callbacks=[
        lgb.early_stopping(
            stopping_rounds=20,
            verbose=10
        )
    ]
)

[LightGBM] [Info] Number of positive: 222, number of negative: 374
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000167 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 189
[LightGBM] [Info] Number of data points in the train set: 596, number of used features: 10
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.372483 -> initscore=-0.521578
[LightGBM] [Info] Start training from score -0.521578
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
Training until validation scores don't improve for 20 rounds
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [W

LGBMClassifier(objective='binary')

In [7]:
oof = gbm.predict(valid_X, num_iteration=gbm.best_iteration_)
print('score', round(accuracy_score(valid_y, oof)*100, 2), '%')

score 81.36 %


In [12]:
test_pred = gbm.predict(test, num_iteration=gbm.best_iteration_)
submission['Survived'] = test_pred
submission.to_csv('submission_holdout.csv', index=False)

In [13]:
from sklearn.model_selection import KFold

In [14]:
kf = KFold(n_splits=3)

score_list = []
models = []

for fold_, (train_idx, valid_idx) in enumerate(kf.split(X_train, y_train)):
    train_X = X_train.iloc[train_idx]
    valid_X = X_train.iloc[valid_idx]
    train_y = y_train[train_idx]
    valid_y = y_train[valid_idx]
    
    print(f'fold{fold_ + 1} start')
    
    gbm = lgb.LGBMClassifier(objective='binary')
    gbm.fit(
        train_X, 
        train_y,
        eval_set=[(valid_X, valid_y)],
        callbacks=[
            lgb.early_stopping(
                stopping_rounds=20,
                verbose=10
            )
        ]
    )
    
    oof = gbm.predict(valid_X, num_iteration=gbm.best_iteration_)
    score_list.append(round(accuracy_score(valid_y, oof)*100, 2))
    models.append(gbm)
    print(f'fold{fold_ + 1} end\n')
print(score_list, 'mean score', np.mean(score_list), '%')

fold1 start
[LightGBM] [Info] Number of positive: 236, number of negative: 358
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000143 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 188
[LightGBM] [Info] Number of data points in the train set: 594, number of used features: 10
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.397306 -> initscore=-0.416701
[LightGBM] [Info] Start training from score -0.416701
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
Training until validation scores don't improve for 20 rounds
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[

In [16]:
test_pred = np.zeros((len(test), 3))

for fold_, gbm in enumerate(models):
    pred_ = gbm.predict(test, num_iteration=gbm.best_iteration_)
    test_pred[:, fold_] = pred_

pred = (np.mean(test_pred, axis=1) > 0.5).astype(int)
submission['Survived'] = pred
submission.to_csv('submission_kfoldout.csv', index=False)

In [17]:
gbm.get_params()

{'boosting_type': 'gbdt',
 'class_weight': None,
 'colsample_bytree': 1.0,
 'importance_type': 'split',
 'learning_rate': 0.1,
 'max_depth': -1,
 'min_child_samples': 20,
 'min_child_weight': 0.001,
 'min_split_gain': 0.0,
 'n_estimators': 100,
 'n_jobs': None,
 'num_leaves': 31,
 'objective': 'binary',
 'random_state': None,
 'reg_alpha': 0.0,
 'reg_lambda': 0.0,
 'subsample': 1.0,
 'subsample_for_bin': 200000,
 'subsample_freq': 0}

In [18]:
from sklearn.model_selection import GridSearchCV

In [19]:
gbm = lgb.LGBMClassifier(objective='binary')

params = {
    'max_depth': [2, 3, 4, 5],
    'reg_alpha': [0, 1, 10, 100],
    'reg_lambda': [0, 1, ]
}

grid_search = GridSearchCV(
    gbm,
    param_grid=params,
    cv=3
)
grid_search.fit(X_train, y_train)

print(grid_search.best_score_)
print(grid_search.best_params_)

[LightGBM] [Info] Number of positive: 228, number of negative: 366
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000178 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 189
[LightGBM] [Info] Number of data points in the train set: 594, number of used features: 10
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.383838 -> initscore=-0.473288
[LightGBM] [Info] Start training from score -0.473288
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf

In [20]:
score_list = []
test_pred = np.zeros((len(test), 3))

for fold_, (train_idx, valid_idx) in enumerate(kf.split(X_train, y_train)):
    train_X = X_train.iloc[train_idx]
    valid_X = X_train.iloc[valid_idx]
    train_y = y_train[train_idx]
    valid_y = y_train[valid_idx]
    
    print(f'fold{fold_ + 1} start')
    
    gbm = lgb.LGBMClassifier(objective='binary', max_depth=3, reg_alpha=1, reg_lambda=0)
    
    gbm.fit(
        train_X, 
        train_y,
        eval_set=[(valid_X, valid_y)],
        callbacks=[
            lgb.early_stopping(
                stopping_rounds=20,
                verbose=-1
            )
        ]
    )
    
    oof = gbm.predict(valid_X, num_iteration=gbm.best_iteration_)
    score_list.append(round(accuracy_score(valid_y, oof)*100, 2))
    test_pred[:, fold_] = gbm.predict(test, num_iteration=gbm.best_iteration_)
    print(f'fold{fold_ + 1} end\n')
    
print(score_list, 'mean score', np.mean(score_list))
pred = (np.mean(test_pred, axis=1) > 0.5).astype(int)
submission['Survived'] = pred
submission.to_csv('submission_grid_search.csv', index=False)

fold1 start
[LightGBM] [Info] Number of positive: 236, number of negative: 358
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000155 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 188
[LightGBM] [Info] Number of data points in the train set: 594, number of used features: 10
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.397306 -> initscore=-0.416701
[LightGBM] [Info] Start training from score -0.416701
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
Training until validation scores don't improve for 20 rounds
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[

In [21]:
import warnings
warnings.filterwarnings('ignore')

In [23]:
train = pd.read_csv('../../csv/titanic/train.csv')
test = pd.read_csv('../../csv/titanic/test.csv')
submission = pd.read_csv('../../csv/titanic/gender_submission.csv')
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [24]:
genders = {'female': 0, 'male': 1}
train['Sex'] = train['Sex'].map(genders)
test['Sex'] = test['Sex'].map(genders)

In [25]:
embarked = {'S': 0, 'C': 1, 'Q': 2}
train['Embarked'] = train['Embarked'].map(embarked)
test['Embarked'] = test['Embarked'].map(embarked)

In [26]:
train = train.drop(['PassengerId', 'Name', 'Cabin', 'Ticket'], axis=1)
test = test.drop(['PassengerId', 'Name', 'Cabin', 'Ticket'], axis=1)

In [27]:
train.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,1,22.0,1,0,7.2500,0.0
1,1,1,0,38.0,1,0,71.2833,1.0
2,1,3,0,26.0,0,0,7.9250,0.0
3,1,1,0,35.0,1,0,53.1000,0.0
4,0,3,1,35.0,0,0,8.0500,0.0


In [28]:
X_train = train.drop('Survived', axis=1)
y_train = train['Survived']

In [29]:
train_X, valid_X, train_y, valid_y = train_test_split(X_train, y_train, test_size=0.3, random_state=42)

gbm = lgb.LGBMClassifier(objective='binary')
gbm.fit(
    train_X, 
    train_y,
    eval_set=[
        (valid_X, valid_y)
    ],
    categorical_feature=[
        'Sex',
        'Embarked'
    ],
    callbacks=[
        lgb.early_stopping(
            stopping_rounds=20,
            verbose=-1
        )
    ]
)

oof = gbm.predict(valid_X, num_iteration=gbm.best_iteration_)
print('Score', round(accuracy_score(valid_y, oof)*100, 2))

[LightGBM] [Info] Number of positive: 231, number of negative: 392
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000083 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 190
[LightGBM] [Info] Number of data points in the train set: 623, number of used features: 7
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.370787 -> initscore=-0.528844
[LightGBM] [Info] Start training from score -0.528844
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
Training until validation scores don't improve for 20 rounds
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Wa

In [30]:
oof = gbm.predict(valid_X, num_iteration=gbm.best_iteration_)
print('score', round(accuracy_score(valid_y, oof)*100, 2))

score 81.34


In [31]:
test_pred = gbm.predict(test, num_iteration=gbm.best_iteration_)
submission['Survived'] = test_pred
submission.to_csv('submission_categorical_gbm.csv', index=False)

In [32]:
gbm.get_params()

{'boosting_type': 'gbdt',
 'class_weight': None,
 'colsample_bytree': 1.0,
 'importance_type': 'split',
 'learning_rate': 0.1,
 'max_depth': -1,
 'min_child_samples': 20,
 'min_child_weight': 0.001,
 'min_split_gain': 0.0,
 'n_estimators': 100,
 'n_jobs': None,
 'num_leaves': 31,
 'objective': 'binary',
 'random_state': None,
 'reg_alpha': 0.0,
 'reg_lambda': 0.0,
 'subsample': 1.0,
 'subsample_for_bin': 200000,
 'subsample_freq': 0}

In [33]:
gbm.feature_importances_

array([ 29,  19, 150,  11,   3, 218,  27], dtype=int32)

In [34]:
pd.DataFrame(
    {
        'feature': X_train.columns,
        'importance': gbm.feature_importances_
    }
).sort_values('importance', ascending=False)

,feature,importance
5,Fare,218
2,Age,150
0,Pclass,29
6,Embarked,27
1,Sex,19
3,SibSp,11
4,Parch,3


In [35]:
gbm = lgb.LGBMClassifier(objective='binary', importance_type='gain')
gbm.fit(
    train_X, 
    train_y,
    eval_set=[
        (valid_X, valid_y)
    ],
    categorical_feature=[
        'Sex',
        'Embarked'
    ],
    callbacks=[
        lgb.early_stopping(
            stopping_rounds=20,
            verbose=-1
        )
    ]
)

[LightGBM] [Info] Number of positive: 231, number of negative: 392
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000096 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 190
[LightGBM] [Info] Number of data points in the train set: 623, number of used features: 7
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.370787 -> initscore=-0.528844
[LightGBM] [Info] Start training from score -0.528844
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
Training until validation scores don't improve for 20 rounds
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Wa

LGBMClassifier(importance_type='gain', objective='binary')

In [36]:
pd.DataFrame(
    {
        'feature': X_train.columns,
        'importance': gbm.feature_importances_
    }
).sort_values('importance', ascending=False)

,feature,importance
1,Sex,968.062619
5,Fare,357.223153
2,Age,337.788552
0,Pclass,331.919812
6,Embarked,75.468469
3,SibSp,30.666792
4,Parch,4.518306
